In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/weather_data/65 Years of Weather Data Bangladesh (1948 - 2013).csv')

In [ ]:
df.head()

,Unnamed: 0,Station Names,YEAR,Max Temp,Min Temp,Rainfall,Relative Humidity,Wind Speed,Cloud Coverage,Bright Sunshine,Station Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Month
0,0,Barisal,1949,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1949.01,1
1,1,Barisal,1950,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1950.01,1
2,2,Barisal,1951,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1951.01,1
3,3,Barisal,1952,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1952.01,1
4,4,Barisal,1953,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1953.01,1


In [ ]:
df.describe()

,Unnamed: 0,YEAR,Max Temp,Min Temp,Rainfall,Relative Humidity,Wind Speed,Cloud Coverage,Bright Sunshine,Station Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Month
count,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000,21120.000000
mean,10559.500000,1985.773295,33.475129,21.148859,197.529667,79.463163,1.415354,3.478883,6.411131,41933.961364,534711.284017,563482.384432,23.362909,90.448776,12.993182,1985.838295,6.500000
std,6096.963179,17.604499,2.986268,4.968361,239.041031,7.664233,1.040356,2.080403,1.739075,37.076717,145302.997111,159680.561697,1.172647,1.125241,13.519959,17.604533,3.452134
min,0.000000,1948.000000,21.600000,6.200000,0.000000,34.000000,0.000000,0.000000,0.000000,41858.000000,0.000000,0.000000,20.870000,88.560000,0.000000,1948.010000,1.000000
25%,5279.750000,1972.000000,31.700000,16.900000,8.000000,75.000000,0.700000,1.500000,5.000000,41907.000000,435303.700000,499110.500000,22.640000,89.360000,4.000000,1972.110000,3.750000
50%,10559.500000,1988.000000,33.900000,23.400000,110.000000,81.000000,1.200000,3.300000,6.782857,41941.000000,540098.600000,544954.500000,23.170000,90.390000,7.000000,1988.020000,6.500000
75%,15839.250000,2001.000000,35.500000,25.400000,310.000000,85.000000,1.900000,5.500000,7.800000,41960.000000,650012.100000,683166.000000,24.290000,91.460000,19.000000,2001.030000,9.250000
max,21119.000000,2013.000000,44.000000,28.100000,2072.000000,97.000000,11.200000,7.900000,11.000000,41998.000000,734765.400000,844822.300000,25.750000,92.260000,63.000000,2013.120000,12.000000


In [ ]:
numeric_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
X = df[numeric_cols].copy()
y = df.Month

In [ ]:
print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))

X.head() # Show first 5 training examples

Shape of input data: (21120, 17) and shape of target variable: (21120,)


,Unnamed: 0,YEAR,Month,Max Temp,Min Temp,Rainfall,Relative Humidity,Wind Speed,Cloud Coverage,Bright Sunshine,Station Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period
0,0,1949,1,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1949.01
1,1,1950,1,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1950.01
2,2,1951,1,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1951.01
3,3,1952,1,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1952.01
4,4,1953,1,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1953.01


In [ ]:
# splitting the data into 10 folds.  
# We will use this 'kf'(KFold splitting stratergy) object as input to cross_val_score() method
kf =KFold(n_splits=10, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 19008, Test set:2112
Fold:2, Train set: 19008, Test set:2112
Fold:3, Train set: 19008, Test set:2112
Fold:4, Train set: 19008, Test set:2112
Fold:5, Train set: 19008, Test set:2112
Fold:6, Train set: 19008, Test set:2112
Fold:7, Train set: 19008, Test set:2112
Fold:8, Train set: 19008, Test set:2112
Fold:9, Train set: 19008, Test set:2112
Fold:10, Train set: 19008, Test set:2112


In [ ]:
df['Station Names'] = pd.to_numeric(df['Station Names'],errors='coerce')

df.drop(['Station Names'], axis = 1)

,Unnamed: 0,YEAR,Max Temp,Min Temp,Rainfall,Relative Humidity,Wind Speed,Cloud Coverage,Bright Sunshine,Station Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Month
0,0,1949,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.70,90.36,4,1949.01,1
1,1,1950,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.70,90.36,4,1950.01,1
2,2,1951,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.70,90.36,4,1951.01,1
3,3,1952,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.70,90.36,4,1952.01,1
4,4,1953,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.70,90.36,4,1953.01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21115,21115,2009,30.3,16.5,0.0,72.0,2.800000,0.9,8.700000,41998,734765.4,308914.1,20.87,92.26,4,2009.12,12
21116,21116,2010,31.7,16.7,193.0,79.0,2.400000,1.4,5.500000,41998,734765.4,308914.1,20.87,92.26,4,2010.12,12
21117,21117,2011,31.5,16.4,0.0,73.0,0.000000,1.5,7.400000,41998,734765.4,308914.1,20.87,92.26,4,2011.12,12
21118,21118,2012,30.0,15.8,0.0,70.0,1.800000,0.2,9.000000,41998,734765.4,308914.1,20.87,92.26,4,2012.12,12


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df.head()


,Unnamed: 0,YEAR,Max Temp,Min Temp,Rainfall,Relative Humidity,Wind Speed,Cloud Coverage,Bright Sunshine,Station Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Month
0,0,1949,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1949.01,1
1,1,1950,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1950.01,1
2,2,1951,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1951.01,1
3,3,1952,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1952.01,1
4,4,1953,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1953.01,1


In [ ]:
X = df.iloc[1:, 1:16].values
y = df.iloc[1:, 16].values

In [ ]:
#label_encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y))

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = seed)

In [ ]:
# fit model training data for xgboost classifier
from sklearn.metrics import confusion_matrix
model = XGBClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
confusion_matrix(y_test, predictions)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.80%


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators = 50)
model1.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn import preprocessing, metrics
print("Accuracy of the test set:",metrics.accuracy_score(y_test,y_pred))

Accuracy of the test set: 0.7679924242424242
